In [1]:
import numpy as np
import time
import math
from datetime import datetime
from datetime import timedelta
import pandas as pd
import requests
import json
import pymongo
import itertools

In [2]:
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Tahoma']

In [3]:
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator

In [4]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["BADS7101"]
q_all_urlList = mydb["SENTIMENT_ALL_SOURCE"]
q_ka_urlList = mydb["SENTIMENT_KAPOOK_API"]
q_da_urlList = mydb["SENTIMENT_DAILYNEWS_API"]

In [5]:
cursor_kp = q_ka_urlList.find()
cursor_dln = q_da_urlList.find()

df_kp = pd.DataFrame(cursor_kp)
df_dln = pd.DataFrame(cursor_dln)

In [6]:
df_kp['hl_sentiment_score'] = df_kp['hl_sentiment_score'].apply(lambda x: 0 if x == '' else x)
df_kp['hl_sentiment_score'] = df_kp['hl_sentiment_score'].apply(lambda x: float(x))
df_kp['t_sentiment_score'] = df_kp['t_sentiment_score'].apply(lambda x: 0 if x == '' else x)
df_kp['t_sentiment_score'] = df_kp['t_sentiment_score'].apply(lambda x: float(x))

In [7]:
df_kp = df_kp[['Category', 'URL', 'headline', 'newsdate', 'totalView',
       'totalShare', 'text', 'hl_sentiment_score', 'hl_polarity', 'hl_neg_kw',
       'hl_pos_kw', 't_sentiment_score', 't_polarity', 't_neg_kw', 't_pos_kw','dateDiff','avg_share_per_day','avg_view_per_day']]
df_dln = df_dln[['Category', 'URL', 'headline', 'newsdate', 'totalView', 
       'totalShare', 'text', 'hl_sentiment_score', 'hl_polarity', 'hl_neg_kw',
       'hl_pos_kw', 't_sentiment_score', 't_polarity', 't_neg_kw', 't_pos_kw','dateDiff','avg_share_per_day','avg_view_per_day']]

In [8]:
df_dln.drop_duplicates(subset='URL', inplace=True, keep='last', ignore_index=True)
df_kp.drop_duplicates(subset='URL', inplace=True, keep='last', ignore_index=True)

In [9]:
df_plotViz = pd.concat([df_kp,df_dln],ignore_index=True)

In [10]:
df_plotViz.shape

(1006, 18)

In [11]:
df_plotViz['avg_share_per_view'] = df_plotViz['avg_share_per_day']/df_plotViz['avg_view_per_day']
df_plotViz['hl_neg_count'] = df_plotViz['hl_neg_kw'].apply(lambda x : len(x))
df_plotViz['hl_pos_count'] = df_plotViz['hl_pos_kw'].apply(lambda x : len(x))
df_plotViz['t_neg_count'] = df_plotViz['t_neg_kw'].apply(lambda x : len(x))
df_plotViz['t_pos_count'] = df_plotViz['t_pos_kw'].apply(lambda x : len(x))

In [12]:
# Rearrange index
dff = pd.DataFrame()
df_plotViz = dff.append(df_plotViz,ignore_index=True)

In [13]:
df_plotViz.head()

,Category,URL,headline,newsdate,totalView,totalShare,text,hl_sentiment_score,hl_polarity,hl_neg_kw,...,t_neg_kw,t_pos_kw,dateDiff,avg_share_per_day,avg_view_per_day,avg_share_per_view,hl_neg_count,hl_pos_count,t_neg_count,t_pos_count
0,Entertain,https://women.kapook.com/view228055.html,เศร้า.. พ่อณรงค์ เตมีรักษ์ คุณพ่อ แต้ว ณฐพร เส...,2020-07-01 11:08:52,58518,2604,คุณพ่อ ณรงค์ เตมีรักษ์ คุณพ่อของ แต้ว ณฐพร เสี...,80.00,negative,"[เศร้า, เสียชีวิต]",...,"[เสียชีวิต, ประสบอุบัติเหตุ, จน, ประสบอุบัติเห...",[],116.0,22.448276,504.465517,0.044499,2,0,8,0
1,Entertain,https://women.kapook.com/view228058.html,เฟี้ยว์ฟ้าว สุดสวิงริงโก้ ไลฟ์โชว์ลอตเตอรี่ ถู...,2020-07-01 12:38:31,39119,3764,อิม เฟี้ยว์ฟ้าว สุดสวิงริงโก้ ไลฟ์โชว์ลอตเตอรี...,75.00,positive,[],...,"[จน, สงสัย, อวด, จน, ขนลุก]","[ดี, เฮง, เฮง]",116.0,32.448276,337.232759,0.096219,0,1,5,3
2,Entertain,https://women.kapook.com/view228070.html,คุณพ่อ แต้ว ณฐพร เสียชีวิต ย้อนรอย อุบัติเหตุ ...,2020-07-02 03:10:35,109784,2013,ย้อนเหตุการณ์ อุบัติเหตุที่คาดไม่ถึงของ พลอากา...,80.00,negative,"[เสียชีวิต, อุบัติเหตุ, จน]",...,"[อุบัติเหตุ, เสียชีวิต, เงียบ, เสียชีวิต, เศร้...","[อย่างเข้มแข็ง, ดูแล, กุศล, ดีขึ้น, อย่างปลอดภ...",115.0,17.504348,954.643478,0.018336,3,1,15,6
3,Entertain,https://women.kapook.com/view228119.html,คุณพ่อ แต้ว ณฐพร อยู่เคียงข้าง แม้สถานะเปลี่ยนไป,2020-07-02 14:32:22,35951,1108,ต้น อาชว์ เดินทางมาร่วมพิธีรดน้ําศพ คุณพ่อ แต้...,0.00,,[เปลี่ยนไป],...,"[เลิกรา, ดราม่า, เสียชีวิต, เปลี่ยนไป]","[เคียงข้าง, คู่รัก, มีมิตร, ดี, เคียงข้าง, ไม่...",115.0,9.634783,312.617391,0.030820,1,1,4,9
4,Entertain,https://women.kapook.com/view228177.html,เปิดเบาะแส พิธีกรเจ้าเสน่ห์ คารมดี เมียสวย แอบ...,2020-07-03 14:07:45,72897,1881,เพจดังปล่อยเบาะแส ข่าวเมาท์ พิธีกรเจ้าเสน่ห์ ค...,88.89,positive,[],...,"[ลับหลัง, ไม่กล้า, ยุ่ง, ไม่ได้, แฉ, ร้าย, เตื...","[คารมดี, สวย, แซ่บ, คารมดี, สวย, แซ่บ, คารมดี,...",114.0,16.500000,639.447368,0.025804,0,2,19,17


In [14]:
def create_ListofDict_InsertMany(df):
    listofdict = []
    for c,lx in enumerate(df.URL,0):
        info = {
            "Category":df.Category[c],
            "URL": lx,
            "headline": df.headline[c],
            "newsdate": df.newsdate[c],
            "totalView": int(df.totalView[c]),
            "totalShare": int(df.totalShare[c]),
            "text": df.text[c],
            "hl_sentiment_score": float(df.hl_sentiment_score[c]),
            "hl_polarity": df.hl_polarity[c],
            "hl_neg_kw": df.hl_neg_kw[c],
            "hl_pos_kw": df.hl_pos_kw[c],
            "t_sentiment_score": float(df.t_sentiment_score[c]),
            "t_polarity": df.t_polarity[c],
            "t_neg_kw": df.t_neg_kw[c],
            "t_pos_kw": df.t_pos_kw[c],
            "dateDiff": float(df.dateDiff[c]), 
            "avg_share_per_day": float(df.avg_share_per_day[c]),
            "avg_view_per_day": float(df.avg_view_per_day[c]),
            "avg_share_per_view": float(df.avg_share_per_view[c]),
            "hl_neg_count": int(df.hl_neg_count[c]),
            "hl_pos_count": int(df.hl_pos_count[c]),
            "t_neg_count": int(df.t_neg_count[c]),
            "t_pos_count": int(df.t_pos_count[c])
        }
        listofdict.append(info)
    return listofdict

In [15]:
df_plotViz = df_plotViz.fillna(0)

In [16]:
q_all_urlList.delete_many({})
process_data = create_ListofDict_InsertMany(df_plotViz)
q_all_urlList.insert_many(process_data)